In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from tqdm import tqdm
import ast
from pathlib import Path
from plinder.data.config import local_config
from plinder.data.cluster.cluster import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
pli_data = pd.read_parquet(local_config['DATA_DIR']/'pli_data_20240409:140625.parquet')
pli_data.columns = [
        col.lower() if "system_ID" != col else col for col in pli_data.columns
    ]
col_dicts = catogorize_columns(pli_data.columns)
for k, v in col_dicts.items():
    print(k, len(v))

entry 15
annotation 28
protein_qcov 68
protein_lddt 136
protein_fident 136
pocket_fident 136
pocket_qcov 69
pocket_lddt 137
pli_qcov 136
ligand 38
others 49


In [12]:
pli_data.shape

(474008, 948)

In [17]:
pdbpath = Path('/Users/xzhang/plinder_local_data/challenge_submission/pdbbind')
pdbbind_nolig = pd.read_csv(pdbpath/'pdbind_nolig.csv')
pdbbind_nolig_or_rec = pd.read_csv(pdbpath/'pdbind_nolig_or_rec.csv')
pdbbind_nolig.shape, pdbbind_nolig_or_rec.shape

((38471, 2), (33928, 2))

In [15]:
col_dicts['others']

['validation_pocket_id',
 'altcode',
 'prox_alternative_configuration_residues_flag',
 'pdb_chain',
 'iridium_pass',
 'joint_pocket_id',
 'plip_pocket_id',
 'ligtype_x',
 'system_biounit',
 'has_pocket',
 'num_pdbs_for_uniprots',
 'has_plip',
 'has_validation',
 'system_number',
 'system_ID',
 'system_protein_chains_count',
 'validity',
 'identical_pocket',
 'pdb_chain_asym',
 'uniprotacs',
 'cath_id',
 'pfam_id',
 'scop2b_id',
 'ecod_t_name_old',
 'panther_id',
 'kinase_name',
 'pdb_chain_auth',
 'ecod_t_name',
 'ligtype_y',
 'rdkit_valid',
 'smiles_nochiral',
 'organic',
 'nha',
 'mw',
 'n_ring',
 'numhacceptors',
 'numhdonors',
 'n_ro_bonds',
 'fcsp3',
 'tanisim_0.26',
 'tanisim_0.5',
 'tanisim_0.7',
 'tanisim_0.99',
 'tanisim_0.95',
 'tanisim_0.9',
 'tanisim_0.85',
 'smiles_id',
 'identity_cluster',
 'diversity_cluster']

In [18]:
pdbbind_nolig = pli_data[pli_data['system_ID'].isin(pdbbind_nolig.system_ID)]
pdbbind_nolig_or_rec = pli_data[pli_data['system_ID'].isin(pdbbind_nolig_or_rec.system_ID)]
pdbbind_nolig.shape, pdbbind_nolig_or_rec.shape 

((52319, 948), (46010, 948))

In [21]:
checklist = ['uniprotacs',
 'cath_id',
 'pfam_id',
 'scop2b_id',
 'panther_id',
 'kinase_name',
 'ecod_t_name']

In [23]:
col = checklist[0]
df = pdbbind_nolig_or_rec
df.loc[~df[col].str.contains("UNK"), col].nunique()

3305

In [54]:
stats = []
for df, name in zip([pdbbind_nolig, pdbbind_nolig_or_rec], ["pdbbind_nolig", "pdbbind_nolig_or_rec"]):
    for col in checklist:
        stats.append([col, name, df.loc[~df[col].str.contains("UNK"), col].nunique()])
for col in checklist:
    df = pli_data
    stats.append([col, 'total num in plinder', df.loc[~df[col].str.contains("UNK"), col].nunique()])
stats = pd.DataFrame(stats, columns=['protein classification', 'dataset', 'num'])
stats.pivot(index='protein classification', columns='dataset', values='num').to_clipboard()

In [43]:
filelst = [
    'gs://plinder-collab-bucket/pre_calculated_cluster_labels/pli_qcov_weighted_sum__0.7__strong__component.csv',
    'gs://plinder-collab-bucket/pre_calculated_cluster_labels/pli_qcov_weighted_sum__0.7__weak__component.csv',

    'gs://plinder-collab-bucket/pre_calculated_cluster_labels/pli_qcov_weighted_sum__0.5__strong__component.csv',
    'gs://plinder-collab-bucket/pre_calculated_cluster_labels/pli_qcov_weighted_sum__0.5__weak__component.csv',

    'gs://plinder-collab-bucket/pre_calculated_cluster_labels/pocket_qcov_weighted_sum__0.7__strong__component.csv',
    'gs://plinder-collab-bucket/pre_calculated_cluster_labels/pocket_qcov_weighted_sum__0.7__weak__component.csv',

    'gs://plinder-collab-bucket/pre_calculated_cluster_labels/pocket_qcov_weighted_sum__0.5__strong__component.csv',
    'gs://plinder-collab-bucket/pre_calculated_cluster_labels/pocket_qcov_weighted_sum__0.5__weak__component.csv',
           ]
dfs = []
for file in tqdm(filelst):
    df = pd.read_csv(file)
    df.set_index('system_ID', inplace=True)
    del df['id']
    # print(df.shape[0])
    dfs.append(df)
dfs = pd.concat(dfs, axis=1).reset_index()

100%|██████████| 8/8 [00:21<00:00,  2.68s/it]


In [48]:
dfs.columns

Index(['system_ID', 'pli_qcov_weighted_sum__0.7__strong__component',
       'pli_qcov_weighted_sum__0.7__weak__component',
       'pli_qcov_weighted_sum__0.5__strong__component',
       'pli_qcov_weighted_sum__0.5__weak__component',
       'pocket_qcov_weighted_sum__0.7__strong__component',
       'pocket_qcov_weighted_sum__0.7__weak__component',
       'pocket_qcov_weighted_sum__0.5__strong__component',
       'pocket_qcov_weighted_sum__0.5__weak__component'],
      dtype='object')

In [55]:
stats2 = []
for df, name in zip([pdbbind_nolig, pdbbind_nolig_or_rec], ["pdbbind_nolig", "pdbbind_nolig_or_rec"]):
    for col in dfs.columns:
        stats2.append([col, name, df[col].nunique()])
for col in dfs.columns:
    df = pli_data
    stats2.append([col, 'total num in plinder', df[col].nunique()])
stats2 = pd.DataFrame(stats2, columns=['similarity cluster', 'dataset', 'num'])
stats2.pivot(index='similarity cluster', columns='dataset', values='num').to_clipboard()

In [56]:
pli_data['ecod_t_name']

0                                           UNK;UNK
1                           TIM barrels;TIM barrels
2                           TIM barrels;TIM barrels
3                           TIM barrels;TIM barrels
4         Fluoride ion channel;Fluoride ion channel
                            ...                    
474003                             Cyclophilin-like
474004                             Cyclophilin-like
474005                                          UNK
474006                                          UNK
474007                           Carbonic anhydrase
Name: ecod_t_name, Length: 474008, dtype: object